In [1]:
from FinMind.data import DataLoader
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv("API_TOKEN")

api = DataLoader()
api.login_by_token(api_token=token)

In [2]:
start_date = "1994-10-02"
days_ago = 20
stock_id = "2385"

candle_low = 0.03
volume_x = 3

tp = 0.05
sl = -0.03

In [3]:
taiwan_stocks = api.taiwan_stock_info()

# Extract stock ids
stock_ids = taiwan_stocks['stock_id'].tolist()

print(f"Total stocks fetched: {len(stock_ids)}")

2024-04-02 22:53:09.568 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockInfo, data_id: 


Total stocks fetched: 3244


In [4]:
import re
pattern = '^[1-9][0-9]{3}$'
filtered_stock_ids = [sid for sid in stock_ids if re.match(pattern, sid)]

In [5]:
filtered_stock_ids

['1101',
 '1102',
 '1103',
 '1104',
 '1107',
 '1108',
 '1109',
 '1110',
 '1201',
 '1203',
 '1210',
 '1213',
 '1215',
 '1216',
 '1217',
 '1218',
 '1219',
 '1220',
 '1225',
 '1227',
 '1229',
 '1230',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1240',
 '1256',
 '1258',
 '1259',
 '1259',
 '1262',
 '1264',
 '1268',
 '1268',
 '1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1311',
 '1312',
 '1313',
 '1314',
 '1315',
 '1316',
 '1319',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1336',
 '1337',
 '1338',
 '1339',
 '1340',
 '1341',
 '1342',
 '1402',
 '1409',
 '1410',
 '1413',
 '1414',
 '1416',
 '1417',
 '1418',
 '1419',
 '1423',
 '1432',
 '1432',
 '1434',
 '1435',
 '1436',
 '1437',
 '1438',
 '1439',
 '1439',
 '1440',
 '1441',
 '1442',
 '1443',
 '1443',
 '1444',
 '1445',
 '1446',
 '1447',
 '1449',
 '1451',
 '1452',
 '1453',
 '1453',
 '1454',
 '1455',
 '1456',
 '1456',
 '1457',
 '1459',
 '1460',
 '1463',
 '1464',
 '1465',
 '1466',
 '1467',
 '1468',
 '1469',
 

In [6]:
import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

In [7]:
# Get the current time as a datetime object
current_datetime = datetime.now()

# Calculate the datetime for 30 days ago
n_days_ago_datetime = current_datetime - timedelta(days=days_ago)

# Format both dates as 'YYYY-MM-DD'
current_date_str = current_datetime.strftime('%Y-%m-%d')
n_days_ago_date_str = n_days_ago_datetime.strftime('%Y-%m-%d')

print(f"Current Date: {current_date_str}")
print(f"Date {days_ago} Days Ago: {n_days_ago_date_str}")

Current Date: 2024-04-02
Date 20 Days Ago: 2024-03-13


In [9]:
df = api.taiwan_stock_daily_adj(
    stock_id = stock_id,
    start_date = start_date,
    end_date = current_date_str,
)

2024-04-02 23:00:08.097 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 2385


In [29]:
df
df.to_csv("all_data.csv")

In [11]:
def trades(df):
    # condition 1
    c1_green = ((df['open'] - df['min']) / df['open']) >= candle_low
    c1_red = ((df['close'] - df['min']) / df['open']) >= candle_low
    c1 = ((df['close'] > df['open']) & c1_green) | ((df['open'] > df['close']) & c1_red)
    
    # condition 2
    rolling_avg_volume = df['Trading_Volume'].rolling(window=days_ago).mean()
    c2 = df['Trading_Volume'] >= rolling_avg_volume * volume_x
    
    # Combine conditions
    conditions_met = c1 & c2
    
    conditions_met_shifted = conditions_met.shift(1, fill_value=False)
    
    df['trade_entry'] = conditions_met_shifted
    df['exit_signal'] = False
    df['next_day_open'] = df['open'].shift(-1)
    
    # Iterate through DataFrame to simulate trade entries and exits
    in_trade = False
    entry_price = None
    
    for i in range(len(df)):
        if not in_trade:
            # Check if we should enter a trade
            if df.iloc[i]['trade_entry']:
                in_trade = True
                entry_price = df.iloc[i]['open']
                df.at[i, 'entry_price'] = entry_price
        else:
            # Calculate price changes since entry
            max_pct_change = (df.iloc[i]['max'] - entry_price) / entry_price
            min_pct_change = (df.iloc[i]['min'] - entry_price) / entry_price
            
            exit_based_on_close = (df.iloc[i]['close'] - entry_price) / entry_price
            
            # Check exit criteria: max reaching 5% or min reaching -3%
            if max_pct_change >= tp or min_pct_change <= sl:
                df.at[i, 'exit_signal'] = True
                df.at[i, 'exit_price'] = entry_price * (1 + tp if max_pct_change >= tp else 1 + sl)
                in_trade = False
                entry_price = None
    
    # Filter to find rows where a trade was exited
    # trades_exited = df[df['exit_signal'] | df['trade_entry']]
    
    # print(trades_exited[['open', 'max', 'min', 'close', 'entry_price', 'exit_price', 'trade_entry', 'exit_signal']])

In [26]:
def trades_ex(df):
    trades_exited = df[df['exit_signal'] | df['trade_entry']]    
    print(trades_exited[['open', 'max', 'min', 'close', 'entry_price', 'exit_price', 'trade_entry', 'exit_signal']]) 
    trades_exited.to_csv("trades_ex.csv")

In [13]:
# Initialize variables for calculating wins and total profits
def win_profit(df):    
    total_trades = 0
    wins = 0
    total_profit = 0.0
    entry_size_ntd = 1000000
    
    for i in df.index[df['exit_signal']]:
        # Create a boolean array up to the current index i, indicating where trade_entry is True
        trade_entries_up_to_i = df['trade_entry'][:i+1]
        if trade_entries_up_to_i.any():
            # Find the last True 'trade_entry' index before the current 'exit_signal'
            entry_index = trade_entries_up_to_i[trade_entries_up_to_i].index[-1]
            entry_price = df.loc[entry_index, 'entry_price']
            exit_price = df.loc[i, 'exit_price']
            shares_bought = entry_size_ntd / (entry_price * 1000)
            profit = (exit_price - entry_price) * shares_bought * 1000
            total_profit += profit
            total_trades += 1
            if profit > 0:
                wins += 1
                df.loc[i, 'win'] = 1
    
    total_profit = int(total_profit)  # Convert total profit to integer
    win_rate = round((wins / total_trades) * 100, 2) if total_trades > 0 else 0  # Round win rate to 2 decimals
    number_of_trades = len(df[df['exit_signal'] == True])
    
    return win_rate, total_profit, number_of_trades

# print(f"Win Rate: {win_rate:.2f}%")
# print(f"Total Profit: {total_profit}")

In [16]:
accumulated_data = []

# Initialize counters for total wins and total trades across all stocks
total_wins = 0
total_trades = 0

In [17]:
for stock_id in filtered_stock_ids:
    try:
        df = api.taiwan_stock_daily_adj(
            stock_id=stock_id,
            start_date=start_date,
            end_date=current_date_str,
        )
        
        trades(df)
        rate, profit, num_trades = win_profit(df)
        print(rate, profit)
        
        total_wins += df['win'].sum()  # Assuming your df has a 'win' column indicating win(1)/loss(0) for each trade
        total_trades += num_trades
                
        # Append the results for the current stock to the consolidated DataFrame
        accumulated_data.append({
            'stock_id': stock_id,
            'win_rate': "{:.2f}%".format(rate),
            'total_profit': int(profit)  # Convert total_profit to integer
        })
        
    except Exception as e:
        print(f"Error processing {stock_id}: {e}")


2024-04-02 23:08:56.361 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 1101
2024-04-02 23:08:59.623 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 1102


66.67 70000


2024-04-02 23:09:02.714 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 1103


25.0 -40000


2024-04-02 23:09:05.590 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 1104


...

2024-04-03 01:07:35.396 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 9960


13.33 -290000


2024-04-03 01:07:38.263 | INFO     | FinMind.data.finmind_api:get_data:125 - download Dataset.TaiwanStockPriceAdj, data_id: 9962


13.33 -290000
Error processing 9962: cannot convert float NaN to integer


NameError: name 'final_win_rate' is not defined

In [18]:
final_win_rate = round((total_wins / total_trades) * 100, 2) if total_trades > 0 else 0
consolidated_results = pd.DataFrame(accumulated_data)
consolidated_results.loc['final'] = ['All Stocks', "{:.2f}%".format(final_win_rate), consolidated_results['total_profit'].astype(int).sum()]

print(consolidated_results)

         stock_id win_rate  total_profit
0            1101   66.67%         70000
1            1102   25.00%        -40000
2            1103   50.00%         19999
3            1104   50.00%        100000
4            1107   62.50%        160000
...           ...      ...           ...
1506         9945   33.33%        -20000
1507         9946   37.50%             0
1508         9960   13.33%       -290000
1509         9960   13.33%       -290000
final  All Stocks   40.64%      28489988

[1511 rows x 3 columns]


In [19]:
consolidated_results = pd.DataFrame(accumulated_data)
print(consolidated_results)
consolidated_results.to_csv("backtest.csv")

     stock_id win_rate  total_profit
0        1101   66.67%         70000
1        1102   25.00%        -40000
2        1103   50.00%         19999
3        1104   50.00%        100000
4        1107   62.50%        160000
...       ...      ...           ...
1505     9944   27.27%        -89999
1506     9945   33.33%        -20000
1507     9946   37.50%             0
1508     9960   13.33%       -290000
1509     9960   13.33%       -290000

[1510 rows x 3 columns]


In [27]:
trades_ex(df)

       open    max    min  close  entry_price  exit_price  trade_entry  \
110   11.50  11.70  11.25  11.39        11.50         NaN         True   
113   11.37  11.43  11.13  11.39          NaN     11.1550        False   
574    2.22   2.46   2.22   2.41         2.22         NaN         True   
575    2.41   2.58   2.41   2.58          NaN      2.3310        False   
605    2.69   2.69   2.63   2.69         2.69         NaN         True   
607    2.59   2.82   2.59   2.64          NaN      2.6093        False   
1307   7.31   7.36   7.23   7.31         7.31         NaN         True   
1311   7.20   7.28   7.09   7.25          NaN      7.0907        False   
1473   5.50   5.68   5.50   5.60         5.50         NaN         True   
1480   5.96   5.96   5.96   5.96          NaN      5.7750        False   
1573   6.18   6.50   6.18   6.50         6.18         NaN         True   
1582   6.56   6.56   6.41   6.41          NaN      6.4890        False   
2422   7.37   7.72   6.96   7.16      